In [1]:
import pandas as pd
import numpy as np
import re
import pickle

In [2]:
df_road_network = pd.read_csv('./data/road_network_sub-dataset.csv',sep='\t')

In [3]:
speed = pd.read_pickle('./data/event_traffic_beijing_1km_mv_avg_15min_completion.pkl')

In [7]:
road_network_linkid_filtered = pd.read_pickle('./data/road_network_linkid_filtered_6392.pkl')
len(road_network_linkid_filtered)

6392

In [8]:
speed_filter = []
for i in road_network_linkid_filtered:
    speed_filter.append(speed[str(i)])
print(np.array(speed_filter).shape)

(6392, 5856)


In [9]:
def compute_graph_sml(data, delta):
    # (num_nodes,total_intervals*num_features)
    # n=num_nodes, c = total_intervals*num_features
    n, c = data.shape  
    graph_sml = np.zeros((n, n), dtype=np.float32)
    for i in range(n):
        for j in range(i, n):
            a = np.linalg.norm(data[i] - data[j])**2
            b = np.minimum(np.linalg.norm(data[i])**2, np.linalg.norm(data[j])**2)
            c = np.exp(-a / b)
            if c > delta:
                graph_sml[j, i] = graph_sml[i, j] = c

    return graph_sml

In [10]:
sml_mat = compute_graph_sml(np.array(speed_filter),0.1)
sml_mat.shape

(6392, 6392)

In [11]:
sml_mat

array([[1.        , 0.92612165, 0.8700232 , ..., 0.9291955 , 0.9580964 ,
        0.9707211 ],
       [0.92612165, 1.        , 0.91465575, ..., 0.9510194 , 0.9083165 ,
        0.9411666 ],
       [0.8700232 , 0.91465575, 1.        , ..., 0.9391923 , 0.87101394,
        0.87446636],
       ...,
       [0.9291955 , 0.9510194 , 0.9391923 , ..., 1.        , 0.9256878 ,
        0.9414523 ],
       [0.9580964 , 0.9083165 , 0.87101394, ..., 0.9256878 , 1.        ,
        0.97430164],
       [0.9707211 , 0.9411666 , 0.87446636, ..., 0.9414523 , 0.97430164,
        1.        ]], dtype=float32)

In [12]:
with open('./data/sml_mat_6392.pkl', 'wb') as file:
    pickle.dump(sml_mat, file)
file.close()

In [13]:
sml_mat = pd.read_pickle('./data/sml_mat_6392.pkl')
sml_mat.shape

(6392, 6392)